# 2. RNA-Seq analysis

## 2.1. Read Prep
1. Go into the terminal and make a new directory on your desktop called “trimming”.

In [ ]:
#change to home directory
cd ~

In [ ]:
#change to Desktop directory
cd Desktop

In [ ]:
#make 'Trimming' directory
mkdir Trimming

In [ ]:
#change to Trimming directory
cd Trimming

In [41]:
#print working directory to make sure you are in the correct directory
pwd

/home/manager/Desktop/Trimming


In [35]:
#link - create short cut to example datafiles, in your Trimming directory
ln -s ~/Desktop/rnaseq/input/Trimming/* .

In [40]:
#list and check to see if you have all required files
ls

Tumor_RNAseq_R1.fastq  Tumor_RNAseq_R2.fastq  adapt.fasta


In [42]:
#look at the fastq file
head Tumor_RNAseq_R1.fastq

@HWI-ST466_135068617:8:1101:1410:13698/1
ACTTTGTGTTTGAGGATAAGAANGANNCACCTNAGGAGNNNNNNGCGGACGTCTTAGCNGAGGNNATNACNGCAGCAGTNAGGGCCGTAGATNGGGAAGG
+
<?@B?DA?C?FAHGICAGAFCE#3<##)1?E?#1:BBG##############################################################
@HWI-ST466_135068617:8:1101:1411:13794/1
TCTCATTGCTACCAGTGGCTTGNTANNAAATGNTAGCANNNNNNTCCGGATACAGAGANATTTNNTGNACNGCATTTATNGTCTTCAGAGATNCAGTTTT
+
CCCFFFFFHHGHHJJEHIJIJJ#2A##1:CCG#1?DHI######0-<FHEHIIJJJIH#-5@D##,,#,;#,;?@BDED#,5?BDDEDDDDC#+2<8?DD
@HWI-ST466_135068617:8:1101:1413:69882/1
ATGAAACCCTCCAGTCTATTATNGTNNCATAGNTAATGNTNNNNCGTGTTTTCTGATTNGGTGNNCTNAGNCTTGGTGCNTCCCTCTCCACANCCCCCAC


In [43]:
#look at the adapter file
head adapt.fasta

>PrefixPE/1
TACACTCTTTCCCTACACGACGCTCTTCCGATCT
>PrefixPE/2
GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT

2. We will run FASTQC on some paired end data and inspect the results (REMEMBER TAB COMPLETION! It prevents typos).

In [36]:
#make fastq directory in your desktop
mkdir ~/Desktop/fastq

In [38]:
#run fastqc on R1 and R2 sample
source /etc/profile.d/markcbm.sh
fastqc Tumor_RNAseq_R1.fastq -o ~/Desktop/fastqc
fastqc Tumor_RNAseq_R2.fastq -o ~/Desktop/fastqc

Started analysis of Tumor_RNAseq_R1.fastq
Approx 5% complete for Tumor_RNAseq_R1.fastq
Approx 10% complete for Tumor_RNAseq_R1.fastq
Approx 15% complete for Tumor_RNAseq_R1.fastq
Approx 20% complete for Tumor_RNAseq_R1.fastq
Approx 25% complete for Tumor_RNAseq_R1.fastq
Approx 30% complete for Tumor_RNAseq_R1.fastq
Approx 35% complete for Tumor_RNAseq_R1.fastq
Approx 40% complete for Tumor_RNAseq_R1.fastq
Approx 45% complete for Tumor_RNAseq_R1.fastq
Approx 50% complete for Tumor_RNAseq_R1.fastq
Approx 55% complete for Tumor_RNAseq_R1.fastq
Approx 60% complete for Tumor_RNAseq_R1.fastq
Approx 65% complete for Tumor_RNAseq_R1.fastq
Approx 70% complete for Tumor_RNAseq_R1.fastq
Approx 75% complete for Tumor_RNAseq_R1.fastq
Approx 80% complete for Tumor_RNAseq_R1.fastq
Approx 85% complete for Tumor_RNAseq_R1.fastq
Approx 90% complete for Tumor_RNAseq_R1.fastq
Approx 95% complete for Tumor_RNAseq_R1.fastq
Analysis complete for Tumor_RNAseq_R1.fastq
Started analysis of Tumor_RNAseq_R2.fastq

3. Now let’s look at those FASTQC results.

In [44]:
#change to fastqc output directory
cd ~/Desktop/fastqc

In [45]:
#open the fastqc html output
firefox Tumor_RNAseq_R1_fastqc.html

4. We are going to use Trimmomatic to trim the reads. Type the long command on a single line. Manual available at http://www.usadellab.org/cms/?page=trimmomatic. 

In [46]:
#change to Trimming directory and run trimmomatic
cd ~/Desktop/Trimming
java -jar /usr/local/molbiocloud/Trimmomatic-0.39/trimmomatic-0.39.jar \
PE -threads 4 \
Tumor_RNAseq_R1.fastq Tumor_RNAseq_R2.fastq \
Tumor_RNAseq_R1_pe.fq Tumor_RNAseq_R1_se.fq \
Tumor_RNAseq_R2_pe.fq Tumor_RNAseq_R2_se.fq \
ILLUMINACLIP:adapt.fasta:2:30:10 \
LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:70

TrimmomaticPE: Started with arguments:
 -threads 4 Tumor_RNAseq_R1.fastq Tumor_RNAseq_R2.fastq Tumor_RNAseq_R1_pe.fq Tumor_RNAseq_R1_se.fq Tumor_RNAseq_R2_pe.fq Tumor_RNAseq_R2_se.fq ILLUMINACLIP:adapt.fasta:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:70
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 27908 Both Surviving: 24897 (89.21%) Forward Only Surviving: 1535 (5.50%) Reverse Only Surviving: 874 (3.13%) Dropped: 602 (2.16%)
TrimmomaticPE: Completed successfully


This will perform the following:
- Remove sequences that match the adapter (ILLUMINACLIP:adapt.fasta:2:30:10)
- Remove leading low quality bases (below quality 3) (LEADING:3)
- Remove trailing low quality bases (below quality 3) (TRAILING:3)
- Scan the read with a 4-base wide sliding window, cutting when the average quality per base drops below 20 (SLIDINGWINDOW:4:20)
- Remove reads less than 70 bases long (MINLEN:70)

5. Now we will run FASTQC on one of the trimmed files

In [47]:
fastqc Tumor_RNAseq_R1_pe.fq -o ~/Desktop/fastqc

Started analysis of Tumor_RNAseq_R1_pe.fq
Approx 5% complete for Tumor_RNAseq_R1_pe.fq
Approx 10% complete for Tumor_RNAseq_R1_pe.fq
Approx 15% complete for Tumor_RNAseq_R1_pe.fq
Approx 20% complete for Tumor_RNAseq_R1_pe.fq
Approx 25% complete for Tumor_RNAseq_R1_pe.fq
Approx 30% complete for Tumor_RNAseq_R1_pe.fq
Approx 35% complete for Tumor_RNAseq_R1_pe.fq
Approx 40% complete for Tumor_RNAseq_R1_pe.fq
Approx 45% complete for Tumor_RNAseq_R1_pe.fq
Approx 50% complete for Tumor_RNAseq_R1_pe.fq
Approx 55% complete for Tumor_RNAseq_R1_pe.fq
Approx 60% complete for Tumor_RNAseq_R1_pe.fq
Approx 65% complete for Tumor_RNAseq_R1_pe.fq
Approx 70% complete for Tumor_RNAseq_R1_pe.fq
Approx 75% complete for Tumor_RNAseq_R1_pe.fq
Approx 80% complete for Tumor_RNAseq_R1_pe.fq
Approx 85% complete for Tumor_RNAseq_R1_pe.fq
Approx 90% complete for Tumor_RNAseq_R1_pe.fq
Approx 95% complete for Tumor_RNAseq_R1_pe.fq
Analysis complete for Tumor_RNAseq_R1_pe.fq


6. Now let’s look at those FASTQC results.

In [48]:
cd ~/Desktop/fastqc
firefox Tumor_RNAseq_R1_pe_fastqc.html

## 2.2 Alignment and transcript assembly
1. Get the data for today’s work

In [52]:
cd ~/Desktop
mkdir hisat
cd hisat
ln -s ~/Desktop/rnaseq/input/hisat/* .
ls

chr16.fa   rna_cntl_1.fastq  rna_expt_1.fastq
chr16.gtf  rna_cntl_2.fastq  rna_expt_2.fastq


In [53]:
#look at the .fa file and .gtf file contents
head chr16.fa

>chr16
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


In [54]:
head chr16.gtf

chr16	hg38_ncbiRefSeq	exon	8379167	8379403	0.000000	+	.	gene_id "XR_933045.1"; transcript_id "XR_933045.1"; 
chr16	hg38_ncbiRefSeq	exon	8392003	8392122	0.000000	+	.	gene_id "XR_933045.1"; transcript_id "XR_933045.1"; 
chr16	hg38_ncbiRefSeq	exon	8403654	8404151	0.000000	+	.	gene_id "XR_933045.1"; transcript_id "XR_933045.1"; 
chr16	hg38_ncbiRefSeq	exon	25111709	25111768	0.000000	+	.	gene_id "XM_005255354.3"; transcript_id "XM_005255354.3"; 
chr16	hg38_ncbiRefSeq	exon	25128475	25128566	0.000000	+	.	gene_id "XM_005255354.3"; transcript_id "XM_005255354.3"; 
chr16	hg38_ncbiRefSeq	start_codon	25132497	25132499	0.000000	+	.	gene_id "XM_005255354.3"; transcript_id "XM_005255354.3"; 
chr16	hg38_ncbiRefSeq	CDS	25132497	25132523	0.000000	+	0	gene_id "XM_005255354.3"; transcript_id "XM_005255354.3"; 
chr16	hg38_ncbiRefSeq	exon	25132402	25132523	0.000000	+	.	gene_id "XM_005255354.3"; transcript_id "XM_005255354.3"; 
chr16	hg38_ncbiRefSeq	CDS	25140171	25140247	0.000000	+	0	gene_id "XM_005255354.3";

2. Run FASTQC for our input files and make sure that there aren’t any major problems.

In [55]:
mkdir fastqc 

#run fastqc on all fastq files and send output to fastqc folder
fastqc *.fastq -o fastqc/

cd fastqc 
firefox rna_cntl_1_fastqc.html

#go one folder above
cd ..

Started analysis of rna_cntl_1.fastq
Approx 5% complete for rna_cntl_1.fastq
Approx 10% complete for rna_cntl_1.fastq
Approx 15% complete for rna_cntl_1.fastq
Approx 20% complete for rna_cntl_1.fastq
Approx 25% complete for rna_cntl_1.fastq
Approx 30% complete for rna_cntl_1.fastq
Approx 35% complete for rna_cntl_1.fastq
Approx 40% complete for rna_cntl_1.fastq
Approx 45% complete for rna_cntl_1.fastq
Approx 50% complete for rna_cntl_1.fastq
Approx 55% complete for rna_cntl_1.fastq
Approx 60% complete for rna_cntl_1.fastq
Approx 65% complete for rna_cntl_1.fastq
Approx 70% complete for rna_cntl_1.fastq
Approx 75% complete for rna_cntl_1.fastq
Approx 80% complete for rna_cntl_1.fastq
Approx 85% complete for rna_cntl_1.fastq
Approx 90% complete for rna_cntl_1.fastq
Approx 95% complete for rna_cntl_1.fastq
Analysis complete for rna_cntl_1.fastq
Started analysis of rna_cntl_2.fastq
Approx 5% complete for rna_cntl_2.fastq
Approx 10% complete for rna_cntl_2.fastq
Approx 15% complete for rna_

3. Let’s build an index for HISAT2.

In [56]:
#create a folder for index
mkdir hisat_index

#create index
hisat2-build chr16.fa chr16_Index

#move index file to index folder
mv chr16_Index* hisat_index/


Settings:
  Output files: "chr16_Index.*.ht2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  chr16.fa
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
  Time to read SNPs and splice sites: 00:00:00
Using parameters --bmax 15338614 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 15338614 --dcv 1024
Constructing suffix-array element generat

4. Next we will use HISAT2 to align our fastq files to the index

In [62]:
hisat2 -p 8 --dta -x hisat_index/chr16_Index -U rna_cntl_1.fastq -S rna_cntl_1.sam --summary-file rna_cntl_1_alignStats.txt
#run the above command, but switch sample names
!!:gs/cntl_1/cntl_2
!!:gs/cntl_2/expt_1
!!:gs/expt_1/expt_2

351453 reads; of these:
  351453 (100.00%) were unpaired; of these:
    9155 (2.60%) aligned 0 times
    327099 (93.07%) aligned exactly 1 time
    15199 (4.32%) aligned >1 times
97.40% overall alignment rate
hisat2 -p 8 --dta -x hisat_index/chr16_Index -U rna_cntl_2.fastq -S rna_cntl_2.sam --summary-file rna_cntl_2_alignStats.txt
372271 reads; of these:
  372271 (100.00%) were unpaired; of these:
    9418 (2.53%) aligned 0 times
    345536 (92.82%) aligned exactly 1 time
    17317 (4.65%) aligned >1 times
97.47% overall alignment rate
hisat2 -p 8 --dta -x hisat_index/chr16_Index -U rna_expt_1.fastq -S rna_expt_1.sam --summary-file rna_expt_1_alignStats.txt
488268 reads; of these:
  488268 (100.00%) were unpaired; of these:
    13480 (2.76%) aligned 0 times
    446679 (91.48%) aligned exactly 1 time
    28109 (5.76%) aligned >1 times
97.24% overall alignment rate
hisat2 -p 8 --dta -x hisat_index/chr16_Index -U rna_expt_2.fastq -S rna_expt_2.sam --summary-file rna_expt_2_alignStats.txt


The flags/options in the command are:
- Use 8 processors/cores (-p 8)
- Report alignments tailored for transcript assembly (--dta)
- Basename for the index (hisat_index/Hg38_Index)
- Name of single-end FASTQ for alignment (-U rna_cntl_1.fastq)
- [Note: use -1 READ_1_FILE -2 READ_2_FILE for paired-end data]
- Name of aligned SAM file for output (-S rna_cntl_1.sam)


6. I like to keep my files organized, so let’s go ahead and create a folder called Sam_aligned and move all the aligned SAM files into it (Commands are case sensitive - if you 

In [63]:
mkdir Sam_aligned
mv *.sam Sam_aligned/

7. Similarly let’s create a folder called AlignStats and move all of the alignment stats into this folder.

In [64]:
mkdir AlignedStats
mv *.txt AlignedStats/

8. The next step is to perform transcript assembly using StringTie. To do that, we need to sort the sam output files generated by HISAT2 by chromosomal co-ordinates and convert them into the bam format.

In [70]:
mkdir Bam_aligned 
samtools view -u Sam_aligned/rna_cntl_1.sam | samtools sort - -o Bam_aligned/rna_cntl_1.sorted.bam
!!:gs/cntl_1/cntl_2
!!:gs/cntl_2/expt_1
!!:gs/expt_1/expt_2


samtools view -u Sam_aligned/rna_cntl_2.sam | samtools sort - -o Bam_aligned/rna_cntl_2.sorted.bam
samtools view -u Sam_aligned/rna_expt_1.sam | samtools sort - -o Bam_aligned/rna_expt_1.sorted.bam
samtools view -u Sam_aligned/rna_expt_2.sam | samtools sort - -o Bam_aligned/rna_expt_2.sorted.bam


 10. Now we’re ready to assemble the transcriptome using StringTie. This can be done as follows

In [77]:
stringtie Bam_aligned/rna_cntl_1.sorted.bam -p 8 -o Assembled_transcripts/rna_cntl_1.gtf 
!!:gs/cntl_1/cntl_2
!!:gs/cntl_2/expt_1
!!:gs/expt_1/expt_2

stringtie Bam_aligned/rna_cntl_2.sorted.bam -p 8 -o Assembled_transcripts/rna_cntl_2.gtf 
stringtie Bam_aligned/rna_expt_1.sorted.bam -p 8 -o Assembled_transcripts/rna_expt_1.gtf 
stringtie Bam_aligned/rna_expt_2.sorted.bam -p 8 -o Assembled_transcripts/rna_expt_2.gtf 


11. Now that we have all the transcripts assembled, Let’s go ahead and merge them into a single GTF file that pools all the transcript files together.

In [79]:
ls Assembled_transcripts/*.gtf > Assembled_transcripts/mergelist.txt
cat Assembled_transcripts/mergelist.txt

Assembled_transcripts/rna_cntl_1.gtf
Assembled_transcripts/rna_cntl_2.gtf
Assembled_transcripts/rna_expt_1.gtf
Assembled_transcripts/rna_expt_2.gtf


In [80]:
stringtie --merge -p 8 -o Assembled_transcripts/stringtie_merged.gtf Assembled_transcripts/mergelist.txt

12. Let’s prep for DE analysis using ballgown

In [83]:
stringtie -e -B -p 8 -G Assembled_transcripts/stringtie_merged.gtf -o ballgown/rna_cntl_1/rna_cntl_1.gtf Bam_aligned/rna_cntl_1.sorted.bam
!!:gs/cntl_1/cntl_2
!!:gs/cntl_2/expt_1
!!:gs/expt_1/expt_2

stringtie -e -B -p 8 -G Assembled_transcripts/stringtie_merged.gtf -o ballgown/rna_cntl_2/rna_cntl_2.gtf Bam_aligned/rna_cntl_2.sorted.bam
stringtie -e -B -p 8 -G Assembled_transcripts/stringtie_merged.gtf -o ballgown/rna_expt_1/rna_expt_1.gtf Bam_aligned/rna_expt_1.sorted.bam
stringtie -e -B -p 8 -G Assembled_transcripts/stringtie_merged.gtf -o ballgown/rna_expt_2/rna_expt_2.gtf Bam_aligned/rna_expt_2.sorted.bam


The flags/options in the command are:
- Process only the reads that map to the genes in the reference GTF (-e)
- Output data in Ballgown-compatible format (-B)
- Use 8 processors/cores (-p 8)
- Name of reference GTF file (-G Assembled_transcripts/stringtie.merged.gtf)

#### Free to use/modify/resuse for personal academic purposes
© 2020 American Academy for Biomedical Informatics, Confidential and Proprietary 